<a href="https://colab.research.google.com/github/TianyiZhang-zzz/SupplyChain/blob/main/sc0306.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
from itertools import combinations

# Given Data
products = {
    "Product": [1, 2, 3, 4],
    "Demand": [1000, 300, 100, 50],
    "Common_Ordering_Cost": [150, 150, 150, 150],
    "Specific_Ordering_Cost": [20, 25, 30, 50],
    "Unit_Cost": [50, 60, 30, 30],
    "Holding_Cost_Rate": [0.15, 0.15, 0.15, 0.15]
}

df = pd.DataFrame(products)

# Compute Total Ordering Cost for each product
df["Total_Ordering_Cost"] = df["Common_Ordering_Cost"] + df["Specific_Ordering_Cost"]

# EOQ Calculation: Q = sqrt(2DS/hC)
df["EOQ"] = np.sqrt((2 * df["Demand"] * df["Total_Ordering_Cost"]) / (df["Holding_Cost_Rate"] * df["Unit_Cost"]))

# Independent Sourcing (Q1)
df["Order_Frequency_Ind"] = df["Demand"] / df["EOQ"]
df["Annual_Ordering_Cost_Ind"] = df["Order_Frequency_Ind"] * df["Total_Ordering_Cost"]
df["Annual_Holding_Cost_Ind"] = (df["EOQ"] / 2) * df["Holding_Cost_Rate"] * df["Unit_Cost"]
total_cost_independent = df["Annual_Ordering_Cost_Ind"].sum() + df["Annual_Holding_Cost_Ind"].sum()

# Same Frequency Sourcing (Q2)
agg_demand = df["Demand"].sum()
agg_ordering_cost = df["Common_Ordering_Cost"].max() + df["Specific_Ordering_Cost"].sum()
agg_unit_cost = (df["Unit_Cost"] * df["Demand"]).sum() / agg_demand  # Weighted average cost

EOQ_same = np.sqrt((2 * agg_demand * agg_ordering_cost) / (df["Holding_Cost_Rate"].mean() * agg_unit_cost))
order_frequency_same = agg_demand / EOQ_same
annual_ordering_cost_same = order_frequency_same * agg_ordering_cost
annual_holding_cost_same = (EOQ_same / 2) * df["Holding_Cost_Rate"].mean() * agg_unit_cost
total_cost_same = annual_ordering_cost_same + annual_holding_cost_same

# Function to calculate cost for a given partition
def calculate_partition_cost(partition):
    total_cost = 0
    for group in partition:
        group_demand = df.loc[group, "Demand"].sum()
        group_ordering_cost = df.loc[group, "Common_Ordering_Cost"].max() + df.loc[group, "Specific_Ordering_Cost"].sum()
        group_unit_cost = (df.loc[group, "Unit_Cost"] * df.loc[group, "Demand"]).sum() / group_demand

        EOQ_group = np.sqrt((2 * group_demand * group_ordering_cost) / (df.loc[group, "Holding_Cost_Rate"].mean() * group_unit_cost))
        order_frequency_group = group_demand / EOQ_group
        annual_ordering_cost_group = order_frequency_group * group_ordering_cost
        annual_holding_cost_group = (EOQ_group / 2) * df.loc[group, "Holding_Cost_Rate"].mean() * group_unit_cost

        total_cost += annual_ordering_cost_group + annual_holding_cost_group
    return total_cost

# Different partition strategies for Q3 (Tailored Aggregation)
partitions = [
    [[0], [1], [2], [3]],  # Independent (same as Q1)
    [[0, 1], [2, 3]],      # Two groups
    [[0, 1], [2], [3]],    # Three groups
    [[0, 1, 2, 3]]         # All together (should match Q2)
]

# Compute cost for each partition
partition_costs = {str(p): calculate_partition_cost(p) for p in partitions}

# Find the best partition (lowest cost)
best_partition = min(partition_costs, key=partition_costs.get)
best_partition_cost = partition_costs[best_partition]

# Display results
print(f"Annual Operational Cost (Independent Sourcing): ${total_cost_independent:.2f}")
print(f"Annual Operational Cost (Same Frequency Sourcing): ${total_cost_same:.2f}")
print(f"Best Tailored Aggregation Partition: {best_partition}")
print(f"Annual Operational Cost (Best Tailored Aggregation): ${best_partition_cost:.2f}")


Annual Operational Cost (Independent Sourcing): $3271.48
Annual Operational Cost (Same Frequency Sourcing): $2445.66
Best Tailored Aggregation Partition: [[0, 1, 2, 3]]
Annual Operational Cost (Best Tailored Aggregation): $2445.66
